# Class that implements Emission Matrix for IBD detection

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import socket as socket
import os as os
import sys as sys
import multiprocessing as mp
import h5py
import allel
import itertools as it

socket_name = socket.gethostname()
print(socket_name)

if socket_name.startswith("compute-"):
    print("HSM O2 Computational partition detected.")
    path = "/n/groups/reich/hringbauer/git/hapBLOCK/"  # The Path on Harvard Cluster
else: 
    raise RuntimeWarning("Not compatible machine. Check!!")

os.chdir(path)  # Set the right Path (in line with Atom default)

print(os.getcwd())
print(f"CPU Count: {mp.cpu_count()}")

compute-e-16-231.o2.rc.hms.harvard.edu
HSM O2 Computational partition detected.
/n/groups/reich/hringbauer/git/hapBLOCK
CPU Count: 28


In [2]:
from python3.loaddata import load_loaddata

### Tested whether loading worked

In [ ]:
l = load_loaddata(l_model="simulated", path="./output/simulated/undermodel/sim5/")
l.path
htsl, p, m = l.load_all_data()

### Test whether multi-individual load works

In [3]:
%%time
l = load_loaddata(l_model="hdf5double", path="./output/simulated/TSI05s05e1/ch3_20cm/sim_ch")
l.iids = ["iid20A", "iid20B"]
htsl, p, m = l.load_all_data()

CPU times: user 59.8 ms, sys: 7.35 ms, total: 67.1 ms
Wall time: 149 ms


### Test loading multiple Individuals

In [54]:
%%time
l = load_loaddata(l_model="hdf5multi", path="./output/simulated/TSI05s05e1/ch3_20cm/sim_ch")
l.iids = ["iid20A", "iid20B", "iid21A", "iid21B"]
htsl1, p1, m= l.load_all_data()

CPU times: user 54.4 ms, sys: 12.8 ms, total: 67.2 ms
Wall time: 190 ms


In [5]:
%%time
iids = np.array([[f"iid{i}A", f"iid{i}B"] for i in range(100)])
iids = np.concatenate(iids)

l = load_loaddata(l_model="hdf5", path="./output/simulated/TSI05s05e1/ch3_20cm/sim_ch")
l.iids = iids
l.iids = ["iid20A", "iid20B"]
htsl1, p1, m1= l.load_all_data()

CPU times: user 46.2 ms, sys: 7.57 ms, total: 53.8 ms
Wall time: 131 ms


In [61]:
np.shape(htsl1)

(4, 38799)

In [63]:
(htsl1 == htsl).all()

True

In [56]:
len(m)

38799